# FINN - Code Generation and Compilation
<font size="3">This notebook should give a more detailed insight into the code generation and compilation within FINN. </font>

## Outline
*

<font size="3">Following showSrc function is used to print the source code of function calls in the Jupyter notebook: </font>

In [1]:
import inspect

def showSrc(what):
    print("".join(inspect.getsourcelines(what)[0]))